In [75]:
import json
from ase import Atoms, Atom
from ase.build import molecule, bulk
from ase.units import Bohr,Rydberg,kJ,kB,fs,Hartree,mol,kcal,J


In [53]:
# experimental_gas_formation_energies from Catmap
# The values needs to be subtracted to get the formation energy corresponding to the calculation.
E_expt = json.load(open('E_expt.json', 'r'))

In [84]:
#CCSD(T), cc-pVTZ
#E_CCSD = {'CH3CN':-3606.268885, 'CH3NO2':-6657.612581, 'CH4':-1100.377219, 'CO2':-5124.647044, 'H2':-31.90082102, 'H2O':-2077.106386,	'HCN':-2538.149174, 'N2':-2976.217813, 'N2H4':-3039.47841, 'NH3':-1536.714696, 'NO':-3529.7765, 'NO2':-5572.901563,	'O2':-4085.2211, 'CH4N2O':-6120.856117}

#CCSD(T), 6-311+G(3df,2pd)
E_CCSD = {'NH3':-56.476104, 'H2O':-76.336385, 'H2':-1.17230, 'O2':-150.128285, 'NO':-129.717390, 'N2':-109.373495}
E_CCSD = {key:val*Hartree for key, val in E_CCSD.items()}

E_CCSD = {k: v for k, v in E_CCSD.items() if 'C' not in k}

In [85]:
def get_E_form(name, method):
    if method in ['expt', 'ccsd']:
        E_dict = E_methods[method]
    else:
        print('Specify correct argument for method ! (expt, ccsd, )')
        
    E_elem = {}
    E_elem['H'] = E_dict['H2']/2
    E_elem['O'] = E_dict['O2']/2
    if 'graphite' in E_dict.keys():
        E_elem['C'] = E_dict['graphite']
    E_elem['N'] = E_dict['N2']/2
    counts = mol_dict[name].symbols.formula.count()    
    E_form = E_dict[name]
    
    #print(counts, E_form)
    for elem in counts:
        #print(counts[elem], E_elem[elem])
        E_form += -counts[elem] * E_elem[elem]
    return E_form 

In [93]:
methods = ['expt', 'ccsd']
E_methods = {'expt':E_expt, 'ccsd':E_CCSD}

mol_dict = {}
for method in methods:
    E_method = E_methods[method]
    for mol in E_method:
        try:
            mol_dict[mol] = molecule(mol)    
        except KeyError:
            pass
        
        if 'graphite' in E_method.keys():
            mol_dict['graphite'] = Atoms('C', [[0,0,0]])

In [94]:
E_form_dict = {'expt':{}, 'ccsd':{}}

for method in methods:
    for mol in E_methods[method]:
        #print(method, mol)
        E_form = get_E_form(mol, method)
        if E_form != 0:
            E_form_dict[method][mol] = get_E_form(mol, method)


In [95]:
{key:{'ccsd':E_form_dict['ccsd'][key], 'expt':E_form_dict['expt'][key]} for key in E_form_dict['ccsd'].keys() }

{'NH3': {'ccsd': -0.8410087021620498, 'expt': -0.8158049333000003},
 'H2O': {'ccsd': -2.7195739477403116, 'expt': -2.7126289694},
 'NO': {'ccsd': 0.911581431816785, 'expt': 0.9424228842000001}}

In [96]:
json.dump(E_form_dict['ccsd'], open('E_form_dict_CCSD.json', 'w'))
json.dump(E_form_dict['expt'], open('E_form_dict_Expt.json', 'w'))

In [97]:
E_form_dict['expt']['CO2']

-4.1280012214